In [1]:
with open('input.txt') as f:
    data = f.read()

In [2]:
import numpy as np
import re
from PIL import Image

In [3]:
class Cave:
    def __init__(self):
        self.grid = None
        self.floor = 0
        
        self.median = 0
        self.median_dist = 0
        
        self.sand_start = None
        self.sand_point = None
        self.sand_falling = False
        self.sand_count = 0
        
        self.completed = False
        
    @staticmethod
    def _parse_point(s):
        x, y = s.split(',')
        return int(x), int(y)
    
    @staticmethod
    def _get_points(p1, p2):
        x1, y1 = p1
        x2, y2 = p2
        if x1 != x2 and y1 == y2:
            x1, x2 = sorted([x1, x2])
            in_between = zip(range(x1, x2+1), [y1] * len(range(x1, x2+1)))
        elif x1 == x2 and y1 != y2:
            y1, y2 = sorted([y1, y2])
            in_between = zip([x1] * len(range(y1, y2+1)), range(y1, y2+1))
            
        return list(in_between)
    
    def generate(self, paths):
        X, Y = np.array(list(
            map(lambda x: (int(x[0]), int(x[1])), re.findall(r'(\d+),(\d+)', paths))
        )).T
        
        self.median = np.median(X).astype(int)
        self.median_dist = np.max(X) - self.median + 1
        self.floor = np.max(Y)
        self.grid = np.zeros((self.floor + 2, 1 + 2*self.median_dist), dtype=str)
        self.grid[:] = ' '
        
        self.sand_start = (0, 500 - self.median + self.median_dist)
        
        for path in paths.splitlines():
            start_points = list(map(self._parse_point, path.split(' -> ')))
            for i in range(1, len(start_points)):
                for x, y in self._get_points(start_points[i-1], start_points[i]):
                    x = x - self.median + self.median_dist
                    self.grid[y, x] = '#'
        
    def update(self):
        if self.completed:
            pass
        elif self.sand_falling:
            y1, x1 = self.sand_point
            if y1 > self.floor:
                self.completed = True
            elif self.grid[y1 + 1, x1] == ' ':
                self.grid[self.sand_point] = ' '
                self.sand_point = (y1 + 1, x1)
                self.grid[self.sand_point] = 'O'
            elif self.grid[y1 + 1, x1 - 1] == ' ':
                self.grid[self.sand_point] = ' '
                self.sand_point = (y1 + 1, x1 - 1)
                self.grid[self.sand_point] = 'O'
            elif self.grid[y1 + 1, x1 + 1] == ' ':
                self.grid[self.sand_point] = ' '
                self.sand_point = (y1 + 1, x1 + 1)
                self.grid[self.sand_point] = 'O'
            else:
                self.sand_falling = False
        else:
            self.sand_point = self.sand_start
            self.sand_falling = True
            self.grid[self.sand_point] = 'O'
            self.sand_count += 1

In [4]:
c = Cave()
c.generate(data)
while not c.completed:
    c.update()
c.sand_count

1073

In [5]:
class Cave:
    def __init__(self):
        self.grid = None
        self.floor = 0
        
        self.median = 0
        self.median_dist = 0
        
        self.sand_start = None
        self.sand_point = None
        self.sand_falling = False
        self.sand_count = 0
        
        self.completed = False
        
    @staticmethod
    def _parse_point(s):
        x, y = s.split(',')
        return int(x), int(y)
    
    @staticmethod
    def _get_points(p1, p2):
        x1, y1 = p1
        x2, y2 = p2
        if x1 != x2 and y1 == y2:
            x1, x2 = sorted([x1, x2])
            in_between = zip(range(x1, x2+1), [y1] * len(range(x1, x2+1)))
        elif x1 == x2 and y1 != y2:
            y1, y2 = sorted([y1, y2])
            in_between = zip([x1] * len(range(y1, y2+1)), range(y1, y2+1))
            
        return list(in_between)
    
    def generate(self, paths):
        X, Y = np.array(list(
            map(lambda x: (int(x[0]), int(x[1])), re.findall(r'(\d+),(\d+)', paths))
        )).T
        
        self.median = np.median(X).astype(int)
        self.median_dist = np.max(X) - self.median + 1
        self.floor = np.max(Y)
        self.grid = np.zeros((self.floor + 3, 1 + 10*self.median_dist), dtype=str)
        self.grid[:] = ' '
        self.grid[-1] = '#'
        
        self.sand_start = (0, 500 - self.median + self.median_dist)
        
        for path in paths.splitlines():
            start_points = list(map(self._parse_point, path.split(' -> ')))
            for i in range(1, len(start_points)):
                for x, y in self._get_points(start_points[i-1], start_points[i]):
                    x = x - self.median + self.median_dist
                    self.grid[y, x] = '#'
        
    def update(self):
        if self.completed:
            pass
        elif self.sand_falling:
            y1, x1 = self.sand_point
            if self.grid[y1 + 1, x1] == ' ':
                self.grid[self.sand_point] = ' '
                self.sand_point = (y1 + 1, x1)
                self.grid[self.sand_point] = 'O'
            elif self.grid[y1 + 1, x1 - 1] == ' ':
                self.grid[self.sand_point] = ' '
                self.sand_point = (y1 + 1, x1 - 1)
                self.grid[self.sand_point] = 'O'
            elif self.grid[y1 + 1, x1 + 1] == ' ':
                self.grid[self.sand_point] = ' '
                self.sand_point = (y1 + 1, x1 + 1)
                self.grid[self.sand_point] = 'O'
            else:
                self.sand_falling = False
                if y1 == 0 and x1 == 500 - self.median + self.median_dist:
                    self.completed = True
        else:
            self.sand_point = self.sand_start
            self.sand_falling = True
            self.grid[self.sand_point] = 'O'
            self.sand_count += 1

In [6]:
c = Cave()
c.generate(data)
while not c.completed:
    c.update()
c.sand_count

24659